# Project 1: Assembling Genomes


   <div class="alert alert-block alert-danger">
    <center>Due: <b>Tuesday, 8 February, 4:59pm</b>.</center> 
   </div>
   
   <div class="alert alert-block alert-warning">
   <center>
       <b>Collaboration and Resource Policy</b>
    </center>
    For this assignment, you are encouraged to work with one other person. Your team must satisfy the constraints mentioned in <a href="https://www.dropbox.com/s/g6z2xncwc4jsplp/csbio-class2-inked.pdf?dl=0">Class 2</a>.
    
   1. You went to different high schools.
   2. You and your partner have different answers to <em>at least one</em> of these questions:
       - What country were you born in?
       - Have you taken any biology courses at UVA?
       - Have you taken cs3102?
       - Have you taken cs4102?
    
We expect most students will have the best learning experience on this assignment by working with a partner, but if you prefer to work alone it is okay to do this assignment on your own.
    
You are permitted (actually _encouraged_) to discuss these problems with anyone you want, including other students in the class. If you do discuss the specific questions in the assignment with anyone other than your assignment partner and the course staff, though, you should list them in the _External resources used_ section below.
    
You are welcome to use any resources you want for this assignment, other than ones that would defeat the purpose of the assignment. This means you should not look at answers or code from any other students in the class (other than your collaboration with your partner), and if you find code that implements the problem you are being asked to do for the assignment, you should not use that code. You should document all external resource you use that are not part of the course materials in the _External resources used_ section below.

**Team submitting this assignment:**  
<div class="alert alert-success">
    <b><em>Sindhu Mente (snm6dhh)</em></b><br>
    <b><em>Shruthi Nyshadham (sn5hnj)</em></b>
</div>

**External resources used:** 
<div class="alert alert-success">
<em>It is not necessary to list the course materials, but if you used any other resources, including discussing problems with students not on your team, list them here.</em>
</div>

In this project, we will explore genome assembly — the process of determining the order of nucleotides from fragmented reads that are produced by sequencing machines. 

Genome assembly maps to a very well defined computer science problem, but can get quite complicated, as problems such as full sequence coverage, finding a good length for reads (the $k$ in $k$-mer), and sequencing errors present challenges for sequencing analysis and accuracy. For the required problems (ones everyone is expected to solve), you will be able to assume perfect coverage and no read errors; for the "challenge" problem (that is considered a bonus, and not something we expect everyone to be able to solve, and perhaps may not even be feasible for anyone to solve) you will not be able to rely on such assumptions.

 <div class="alert alert-block alert-warning">
    
<b>Submission</b>: You should work on this assignment by forking the provided Project 1 repository (you probably already did this, following the instructions posted at [https://computingbiology.github.io/project1](https://computingbiology.github.io/project1). Add the other teammate to as a collaborator with write access (also under the Settings tab) to this repository.
    
You should answer the questions and write your code in this Jupyter Notebook. (We don't expect you to need to use any external files or organize your code outside of the notebook, but if you do, make sure to put everything needed to run your code in your repository.) Parts where you are expected to provide and answer (which could be text that can be written in markdown format in the notebook or Python code that runs in the notebook) are marked in green.
        
When you are ready to submit the assignment, you should create a release of the version you are submitting ([github's directions for how to create a release](https://docs.github.com/en/repositories/releasing-projects-on-github/managing-releases-in-a-repository)) tagged as `submit`. After you've done this, send a message in slack to a channel that includes both team members (so the one sending this message should include the other team member) and all of the course staff (`dave`, `Hyun Jae Cho`, and `Anshuman Suri`) and a link to your repository (feel free to add any other useful comments if you want, but the link to your submission repository is sufficient).

## Getting Started

1. Install basic required packages, should be run only once. You may need to restart the jupyter python kernel (under the Kernel menu) after this. (You can execute this directly in the notebook but running the command below.)

In [ ]:
%pip install -r requirements.txt

2. Make sure you have [graphviz](https://graphviz.org/) installed on your system. (On a Mac OS X, `brew install graphviz`. For other platforms, see [_https://graphviz.org/download/_](https://graphviz.org/download/).)

In [ ]:
import collections
import matplotlib.pyplot as plt

import utils

## Part 1: Assembling the TeleTubby Genome

For this part, you're given reads generated while trying to sequence the DNA of a mysterious unknown organism with a _tiny_ genome. Some of the course staff is not sure what a TeleTubby is, but I've been assured that no cute creatures were harmed in producing this data (which was generated synthetically).

By answering the following questions, you will learn how to assemble the original genome sequence from sequence reads.

Sequencing data is often stored in the FASTQ file format, with is a simple ASCII format that is somewhat human-readable. 

In _TeleTubby.fastq_ (which is included in the repository you forked), you will find the data that was read from the TeleTubby genome. Each four lines of the file repeat the same pattern. For example,
```
@TeleTubby Genome: Project 1
TAAAATGG
+
HAICDF5I
```
The first line contains the metadata that encodes the name of the read, the experiment type, the kind of sequencing machine used, etc. 

The second line is the sequence of bases that was read. 

The third line is just a placeholder.

The fourth line is a sequence of base qualities that encodes the qualities for the corresponding bases in the sequence line. We will discuss and use this for Problem 2.

Read in the data from `TeleTubby.fastq`:

In [ ]:
# Read sequence reads (error-free) from file
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')

### Melting Temperature

The GC-content (or the ratio of G and C nucleotides) is related to the melting temperature of a DNA double helix structure. 

The following equation can be used to estimate the melting temperature (in degrees Celsius) of DNA for a particular sequence:

\begin{equation*}
t_m = 64.9+0.41(\%GC)-\frac{500}{\text{length of sequence}}
\end{equation*}

As a reference, the human genome is known to have between 35%-60% GC-content. 

<div class="alert alert-success">

**Problem 1.** Calculate the melting temperature for the TeleTubby genome using the formula above. Assume that the sequence is 300 nucleotides long, and that the provided read data has equal coverage everywhere so the G-C distribution in the reads is very close to the actual G-C distribution for the genome. 
    </div>

In [ ]:
# Write code here for calculating the %GC content from the sequence_reads, and computing the estimated melting temperature.
total_num_bases = 0
num_gc_bases = 0
for read in sequence_reads:
    total_num_bases += len(read)
    for base in read:
        if base == 'G' or base == 'C':
            num_gc_bases += 1
gc_percentage = num_gc_bases / total_num_bases * 100
print("GC%%: %f" % (gc_percentage))

len_of_sequence = 200
melting_temp = 64.9 + (0.41 * gc_percentage) - (500/len_of_sequence)
print("Melting temperature: %f°C" % (melting_temp))

In [ ]:
# Print out temperature in Celsius
print("Melting temperature: %f°C" % (melting_temp))

## Interpreting Read Quality

Phred33 quality scores are represented as the character with an ASCII code equal to its value + 33 (to make them easy to print alongside genome sequences). The Phred scores $Q$ are related to error probabilities $P$ as: $Q = -10 \log_{10}(P)$. The table linked here provides a mapping from the Phred values provided in a fastq file and the $Q$ values: [Quality Score Encoding](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm) 

<div class="alert alert-success">
    
**Problem 2.** Using the Phred scores, identify the _least likely to be correct_ read in the TeleTubby.fastq data, which is the read with the lowest probability that it contains _no_ errors.

</div>

In [ ]:
# Write your code for Problem 2 here. 
# It should output the read (nucleotide sequence) with the lowest probability of containing no errors.
#print(qualities)

quality_sums = []
for read in qualities:
    q_sum = 0
    for char in read:
        q = ord(char) - 33
        q_sum += q
    quality_sums.append(q_sum)

min_quality = min(quality_sums)
#print(min_quality)
min_index = quality_sums.index(min_quality)
#print(min_index)
print(sequence_reads[min_index])

Note: For the following problems, you can ignore the read quality scores, and safely assume there are no actual errors in the provided TeleTubby reads.

## Frequency analysis

Looking at repetitions in the sequence can be helpful in estimating the "redudancy" in the organisms. Eucaryote genomes have a lot of redundancy, while smaller organisms like bacteria have highly packed genomes. One heuristic to estimate this before actually performing the assembly could be looking at how often certain $k$-mers are repeated.

<div class="alert alert-success">
    
**Problem 3.**
    Print out the 3 most frequent <em>k</em>-mers in the TeleTubby reads with their frequencies. (As in Problem 1, doing this from the reads only produces the same result as from the genome if the read coverage is equal everywhere, but it should be a close estimate.)
</div>

In [ ]:
# Your code here
freq = {}
for read in sequence_reads:
    if read in freq:
        freq[read] += 1
    else:
        freq[read] = 1
sorted_keys = sorted(freq, key=freq.get, reverse=True)
for key in sorted_keys[0:3]:
    print("%s: %s" % (key, freq[key]))

## Greedy Assembly

Given a set of sequence fragments, the objective of assembly can be viewed as finding the shortest sequence that contains all the fragments.

One of the approaches to assemble the genome from the given reads is a greedy algorithm:

```
while (len(fragments) > 1):
   calculate pairwise alignments of all pairs of fragments
   merge the two fragments with the largest overlap
genome = fragment[0] # the single remaining fragment is the genome
```


<div class="alert alert-success">
    
**Problem 4 (a)**. What would the runtime be of this algorithm, given $n$ $k$-mer reads? (If you are not sure how to express your answer to this question, review [_Cost of Computation_](https://computingbiology.github.io/complexity/).)
    </div>

<i>Answer</i>: Θ(n^3)
<br>The outer while loop O(n) times, as each round through the loop decreases the number of fragments remaining by 1. The code inside the while loop runs O(n^2) times, as each fragment has to be compared with every other fragment to calculate all the pairwise overlaps. Then to find the two with the biggest overlap, the merge operation has to run O(n) times, making the overall inner code O(n^2). This makes the total function Θ(n^3).

<div class="alert alert-success">

**Problem 4 (b)**. Is this algorithm guaranteed to find the correct genome? (A good answer will define what _correct_ means here, and explain why the algorithm is or is not guaranteed to find it.)

<i>Answer</i>: A <i>correct</i> genome would be a genome sequence that is the same as the actual DNA sequence found biologically in the organism. This algorithm is not guaranteed to find this correct genome, because if there are repeated sequences in the actual DNA sequence that should not be overlapped, this algorithm would likely incorrectly overlap them, resulting in an incorrect final sequence that is missing some of the natural repeats in the real, biological genome.

## Graph-based Assembly

Graphs for genome assembly can be constructed in two ways:

- de Bruijn graph: Processing $k-$mers as edges, with $(k-1)-$mers as nodes, and
- Overlap graph: Processing $k-$mers as nodes, with $(k-1)-$mers as edges.

A de Bruijn graph can be processed to find Euler paths, while Overlap graphs can be processed to find Hamiltonian paths. Both of these methods can be used reconstruct the original genome.

<div class="alert alert-success">
    
**Problem 5.**  Use one of these two techniques to reconstruct the TeleTubby genome from the provided sequence reads. 
    
</div>
    
We have provided some template code below that may be helpful (but feel free to ignore this is you prefer).

In [ ]:
# Read reads into graph

def build_graph(k_mers):
    k_mers = list(set(k_mers))
    edges = []
    nodes = set()
    prefix_len = len(k_mers[0]) - 1
    
    for read in k_mers:
        nodes.add(read[0:prefix_len])
        nodes.add(read[1:prefix_len+1])
        edges.append([read[0:prefix_len], read[1:prefix_len+1]])
        
    return nodes, edges

In [ ]:
#verify that the graph is nearly balanced, and find the start/end nodes that are unbalanced
#also set up graph as adjacency list
def check_balances(nodes, edges):
    start_freq = {}
    end_freq = {}
    edges_remaining = {}

    for edge in edges:
        start_node = edge[0]
        end_node = edge[1]
        if start_node in start_freq:
            start_freq[start_node] += 1
        else:
            start_freq[start_node] = 1
        if end_node in end_freq:
            end_freq[end_node] += 1
        else:
            end_freq[end_node] = 1

        #set up edges_remaining as adjacency list
        if start_node in edges_remaining:
            adj_list = edges_remaining[start_node]
            adj_list.append(end_node)
            edges_remaining[start_node] = adj_list
        else:
            adj_list = []
            adj_list.append(end_node)
            edges_remaining[start_node] = adj_list

    
    #find start
    larger_outdegree = []
    for node in start_freq.keys():
        if node not in end_freq.keys():
            larger_outdegree.append(node)
        elif start_freq[node] > end_freq[node]:
            larger_outdegree.append(node)
    
    #find end
    larger_indegree = []
    for node in end_freq.keys():
        if node not in start_freq.keys():
            larger_indegree.append(node)
        elif start_freq[node] < end_freq[node]:
            larger_indegree.append(node)

    print(larger_outdegree)
    print(larger_indegree)
    
    del start_freq
    del end_freq
    
    return edges_remaining, larger_outdegree[0], larger_indegree[0]

In [ ]:
#find eulerian path
def find_eulerian_path(nodes, edges, start_node, end_node):
    
    #add edge between start and end nodes to create Eulerian cycle
    adj_list = []
    adj_list.append(start_node)
    edges[end_node] = adj_list
    
    print("STARTS")
    
    #get the first cycle
    visited = set()
    cycle = find_cycle(nodes, edges, start_node, start_node, visited)
    print("FIRST CYCLE")
    print(len(cycle))
    
    #remove each edge used in the cycle from remaining edges
    for i in range(0, len(cycle) - 1):
        node = cycle[i]
        adj_list = edges[node]
        adj_list.remove(cycle[i+1])
        if len(adj_list) >= 1: #there are still outgoing edges from that node
            edges[node] = adj_list
        else: #no more outgoing edges from that node
            del edges[node]
    
    #while there are unvisited edges, continue building cycles
    while edges:
        #find newStart node with unexplored edges that is part of cycle
        new_start = None
        for node in cycle:
            if node in edges:
                new_start = node
                break
        
        #re-traverse old cycle from the new start node
#         new_start_index = cycle.index(new_start)
#         new_cycle = []
#         for i in range(new_start_index, len(cycle) - 1):
#             new_cycle.append(cycle[i])
#         for i in range(0, new_start_index):
#             new_cycle.append(cycle[i])
#         cycle = new_cycle
        
        new_start_index = cycle.index(new_start)
        new_cycle = cycle[new_start_index:(len(cycle) - 1)] + cycle[0:new_start_index]
        cycle = new_cycle
        
        #get next cycle
        visited = set()
        new_cycle = find_cycle(nodes, edges, new_start, new_start, visited)
        print("INNER CYCLE")
        print(len(new_cycle))
        
        #remove each edge used in the cycle from remaining edges
        for i in range(0, len(new_cycle) - 1):
            node = new_cycle[i]
            adj_list = edges[node]
            adj_list.remove(new_cycle[i+1])
            if len(adj_list) >= 1: #there are still outgoing edges from that node
                edges[node] = adj_list
            else: #no more outgoing edges from that node
                del edges[node]
        
        #grow cycle to encompass new cycle
        cycle += new_cycle
    
    print("FOUND EULERIAN CYCLE")
    #from cycle, extract Eulerian path        
    path = []
    true_start_index = cycle.index(start_node)
    for i in range(true_start_index, len(cycle) - 1):
        #print(cycle[i], end=" ")
        path.append(cycle[i])
    for i in range(0, true_start_index):
        #print(cycle[i], end=" ")
        path.append(cycle[i])
    return path

In [ ]:
#find eulerian cycle in given graph
def find_cycle(nodes, edges, start, current, visited):
    print(current)
    
    if current == start and current in visited: #if we have found the start again (but skip this initially)
        cycle = []
        cycle.append(current)
        return cycle
    
    else:
        visited.add(current)
        if current in edges: #if there are outgoing edges
            for adj_node in edges[current]: #try every possible outgoing edge to an unvisited node
                if adj_node not in visited or adj_node == start:
                    cycle = find_cycle(nodes, edges, start, adj_node, visited)
                    if cycle != []: #if have found the desired cycle, return the cycle; otherwise, keep trying
                        cycle.insert(0, current)
                        return cycle
        return []

In [ ]:
#define main assembly algorithm
def assemble_sequence(nodes, edges, start, end):
    seq = ""
    
    #find eulerian path
    path = find_eulerian_path(nodes, edges, start, end)
    
    #assemble genome from eulerian path
    seq += path[0]
    del path[0]
    for suffix in path:
        seq += suffix[-1]
    return seq

In [ ]:
# Output assembled sequence
# Hint: Sequence is 200 nucleotides long
nodes, edges = build_graph(sequence_reads)
edges_remaining, start, end = check_balances(nodes, edges)
assembled_seq = assemble_sequence(nodes, edges_remaining, start, end)
print("Sequence: %s" % (assembled_seq))
print("Sequence length: %d" % len(assembled_seq))

In [ ]:
# Needed to properly visualize graphs
%load_ext gvmagic

In [ ]:
# Helper function to visualize graphs
%dotstr utils.viz_graph(nodes, edges)

<div class="alert alert-success"> 
    
**Problem 6.** Which of the two assembly methods did you use for problem 5, and why? (Hint: consider how the costs scale with the number of reads.)
    
</div>

<i>Answer</i>: We used a de Bruijn graph, and found an Eulerian path through that graph, in order to assembly the genome. This was done because it grows much more efficiently for large numbers of reads compared to finding a Hamiltonian path through the overlap graph.

## Part 2: Sequencing SARS-CoV-2 virus

Let's move on from TeleTubbies to real-world organisms. For this problem, you'll assemble a genome for a variant of the SARS-CoV-2 virus. You're given reads from <i>actual</i> genome sequencing runs in the provided `SARS-CoV2.fastq` file. The file is based on the first Covid genome that was submitted on 5 January 2020: [https://www.ncbi.nlm.nih.gov/nuccore/NC_045512](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512) But, you should be reconstructing a sequence from the provided reads without looking at the source sequence.

<div class="alert alert-success">
    
**Problem 7.**  Reconstruct the SATS-CoV2 genome from the provided sequence reads using $k=25$. 
    
</div>

You can re-use your implementation from Problem 5 and simply run it on the new data (depending on what you did for Problem 5, that might be enough to solve this problem). 

Print out your reconstructed sequence to a file `output.txt` (and add this file to the repo you submit). For this part, we will still assume that all the reads are error-free. 

In [ ]:
# Read sequence reads
sequence_reads_covid, qualities_covid = utils.read_fastq('SARS-CoV2.fastq')

In [ ]:
# Read reads into graph
nodes_covid, edges_covid = build_graph(sequence_reads_covid)
edges_remaining_covid, start_covid, end_covid = check_balances(nodes_covid, edges_covid)
del edges_covid

In [ ]:
print(len(sequence_reads_covid))
print(len(nodes_covid))

In [ ]:
#call main assembly algorithm
assembled_covid_seq = assemble_sequence(nodes_covid, edges_remaining_covid, start_covid, end_covid)

In [ ]:
# Write assembled sequence to file
with open("output.txt", "w") as f:
    f.write(assembled_covid_seq)

# Part 3: Error-Aware Assembly (Challenge Problem)

<div class="alert alert-warning">
    This problem is a "Challenge Problem". This means it is a problem of unknown difficulty that might be quite challenging (unlike the earlier problems, we don't have a reference solution for this one, or a clear idea how hard it might be). We do hope all students will at least attempt this and that more ambitious students will work hard to solve it and learn interesting things by the attempt (whether or not it is successful), but not get frustrated if you can't get to the desired answer.  As a "Challenge Problem" it means that you shouldn't be worried if you are not able to solve this, though, and you can get full expected credit on this assignment without answering it.
</div>


In the parts above, we assumed error-free reads while assembling $k$-mers. As much as we'd like that, actual reads can (and do) have errors, captured by their Phred scores. 

For this question, you're given raw, actual reads from sequencing runs. Download the reads from this file:
https://sra-pub-sars-cov2.s3.amazonaws.com/sra-src/SRR11528307/ABS2-LN-R1_cleaned_paired.fastq.gz.  


<div class="alert alert-success">
    
**Problem 8 (Challenge).** Give the reads in the linked fastq file above, including their Phred33 quality score, assemble the most likely genome. Your solution should output the assembled sequence in `challenge.txt`. Provide a brief explantion of how your algorithm works and interesting things you learned in developing it.
    
</div>
    
This is an open-ended question. You are free to use any approach to deal with the issue. Make sure you provide your code, along with any assumptions you may have.

_Write a description of your algorithm, and things you learned from working on this here._

In [ ]:
# implementation 

   <div class="alert alert-block alert-danger">
    <center>
        
 **Remember to follow the submission directions above to submit your assignment by 4:59pm on Tuesday, 8 February.**
    
 </center>
 </div>